In [1]:
import pandas as pd

In [2]:
spec_sample = pd.read_csv("data/zoo2MainSpecz.csv")
phot_sample = pd.read_csv("data/zoo2MainPhotoz.csv")
SDSS = pd.read_csv("data/gz2sample.csv")
image_mapping = pd.read_csv("data/3565489/gz2_filename_mapping.csv")

In [3]:
# Merge spec_sample with SDSS on the dr7objid and OBJID columns
spec_sample = spec_sample.merge(SDSS[['OBJID', 'REDSHIFT', 'REDSHIFTERR',
                                      'PETROR50_R', 'PETROR90_R', 'PETROMAG_U',
                                      'PETROMAG_G', 'PETROMAG_R', 'PETROMAG_I',
                                      'PETROMAG_Z', 'PETROMAGERR_U', 'PETROMAGERR_G',
                                      'PETROMAGERR_R', 'PETROMAGERR_I', 'PETROMAGERR_Z']], 
                                left_on='dr7objid', 
                                right_on='OBJID', 
                                how='left')

phot_sample = phot_sample.merge(SDSS[['OBJID', 'PETROR50_R', 'PETROR90_R',
                                      'PETROMAG_U', 'PETROMAG_G', 'PETROMAG_R', 'PETROMAG_I',
                                      'PETROMAG_Z', 'PETROMAGERR_U', 'PETROMAGERR_G',
                                      'PETROMAGERR_R', 'PETROMAGERR_I', 'PETROMAGERR_Z']], 
                                left_on='dr7objid', 
                                right_on='OBJID', 
                                how='left')

# Drop the redundant OBJID column after merging
spec_sample.drop(columns=['OBJID'], inplace=True)
phot_sample.drop(columns=['OBJID'], inplace=True)

In [4]:
# Merge spec_sample with SDSS on the dr7objid and OBJID columns
spec_sample = spec_sample.merge(image_mapping[['objid', 'asset_id']], 
                                left_on='dr7objid', 
                                right_on='objid', 
                                how='left')

phot_sample = phot_sample.merge(image_mapping[['objid', 'asset_id']], 
                                left_on='dr7objid', 
                                right_on='objid', 
                                how='left')

# Drop the redundant objid column after merging
spec_sample.drop(columns=['objid'], inplace=True)
phot_sample.drop(columns=['objid'], inplace=True)

In [5]:
spec_sample.to_csv("data/spec_sample.csv", index=False)
phot_sample.to_csv("data/phot_sample.csv", index=False)

In [7]:
list(spec_sample)


['specobjid',
 'dr8objid',
 'dr7objid',
 'ra',
 'dec',
 'rastring',
 'decstring',
 'sample',
 'gz2class',
 'total_classifications',
 'total_votes',
 't01_smooth_or_features_a01_smooth_count',
 't01_smooth_or_features_a01_smooth_weight',
 't01_smooth_or_features_a01_smooth_fraction',
 't01_smooth_or_features_a01_smooth_weighted_fraction',
 't01_smooth_or_features_a01_smooth_debiased',
 't01_smooth_or_features_a01_smooth_flag',
 't01_smooth_or_features_a02_features_or_disk_count',
 't01_smooth_or_features_a02_features_or_disk_weight',
 't01_smooth_or_features_a02_features_or_disk_fraction',
 't01_smooth_or_features_a02_features_or_disk_weighted_fraction',
 't01_smooth_or_features_a02_features_or_disk_debiased',
 't01_smooth_or_features_a02_features_or_disk_flag',
 't01_smooth_or_features_a03_star_or_artifact_count',
 't01_smooth_or_features_a03_star_or_artifact_weight',
 't01_smooth_or_features_a03_star_or_artifact_fraction',
 't01_smooth_or_features_a03_star_or_artifact_weighted_fractio

In [ ]:
spec_sample['gz2class'].value_counts()[0:20]

In [10]:
import glob
import os
import pandas as pd

In [48]:
df = pd.read_csv("data/spec_sample.csv")

In [49]:
import os
missing_asset_ids = []

for index, row in df.iterrows():
    file_path = f"data/3565489/images/{row['asset_id']}.jpg"
    if os.path.exists(file_path):
        pass
        # print(f"File exists: {file_path}")
    else:
        # print(f"{file_path}", row['asset_id'])
        missing_asset_ids.append(row['asset_id'])
        
print(len(missing_asset_ids) / len(df))

0.0


In [44]:
df = df[~df['asset_id'].isin(missing_asset_ids)]

In [47]:
df

,dr8objid,dr7objid,ra,dec,rastring,decstring,sample,gz2class,total_classifications,total_votes,...,PETROMAG_G,PETROMAG_R,PETROMAG_I,PETROMAG_Z,PETROMAGERR_U,PETROMAGERR_G,PETROMAGERR_R,PETROMAGERR_I,PETROMAGERR_Z,asset_id
0,1.237649e+18,587722981736579107,171.990940,-1.214009,11:27:57.82,-01:12:50.4,original,Ec,43,181,...,16.808289,15.780492,15.409781,15.245371,0.110597,0.169874,0.130950,0.186950,0.226374,2
1,1.237649e+18,587722981741691055,183.749980,-1.144377,12:14:60.00,-01:08:39.8,original,Er,44,133,...,18.242435,16.978014,16.517690,16.162617,0.237573,0.021458,0.010632,0.010693,0.022354,10
2,NaN,587722981745819655,193.091950,-1.199559,12:52:22.07,-01:11:58.4,original,Sb(o),46,221,...,11.282819,10.430051,9.952703,9.613507,0.004258,0.003033,0.002454,0.002069,0.001532,62
3,NaN,587722981746082020,193.789280,-1.091569,12:55:09.43,-01:05:29.6,original,Sb(o),44,172,...,17.680758,16.733738,16.332094,15.939197,0.163807,0.052897,0.049467,0.064485,0.052755,64
4,1.237649e+18,587722981746344092,194.342180,-1.057973,12:57:22.12,-01:03:28.7,original,SBc2m,43,358,...,17.018332,16.330818,15.965699,15.704073,0.051322,0.009214,0.006835,0.007292,0.019365,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42457,1.237660e+18,588015510369009724,59.228817,1.024012,03:56:54.92,+01:01:26.4,stripe82,SBc2l,53,299,...,19.184055,17.505611,16.806099,16.348060,0.499286,0.097914,0.016225,0.018778,0.041520,295299
42458,1.237660e+18,588015510369075314,59.363860,0.852723,03:57:27.33,+00:51:09.8,stripe82,Ei,48,148,...,18.224540,17.044369,16.442926,15.994728,0.395641,0.025767,0.011791,0.011451,0.020336,295300
42459,1.237660e+18,588015510369075404,59.353092,0.918880,03:57:24.74,+00:55:08.0,stripe82,Er,48,140,...,18.934090,17.555395,16.943670,16.498877,0.308386,0.067847,0.030572,0.028615,0.038565,295301
42460,1.237660e+18,588015510369140934,59.408580,0.881916,03:57:38.06,+00:52:54.9,stripe82,Ei,45,176,...,18.939768,17.596500,16.992682,16.469715,0.728212,0.066374,0.017840,0.018744,0.039002,295302


In [ ]:
df.to_csv("data/phot_sample.csv", index=False)